In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
#hide
import json
from pathlib import Path
from nbdev.showdoc import show_doc
from getpass import getuser, getpass
from fiscaliza.redmine import *
from fiscaliza.constants import SERVICOS
from rich.console import Console
console = Console()

# Fiscaliza

> Scripts para baixar informações sobre inspeções no Sistema Fiscaliza da Anatel e atualizar inspeções por meio da API do Redmine.

## Instalação
Como boa prática é sugerido a criação de um ambiente virtual para encapsular o ambiente python no qual você irá instalar a biblioteca

Utilizando o `conda`:
```bash
conda create -n fiscaliza python=3.8
conda activate fiscaliza
python -m pip install fiscaliza
```
Utilizando somente python:

Windows
```bash
python -m venv <pasta onde criar o ambiente virtual>
<pasta onde criou o ambiente virtual>\Scripts\activate.bat
python -m pip install fiscaliza
```
Linux:
```bash
python -m venv <pasta onde criar o ambiente virtual>
source <pasta onde criou o ambiente virtual>/Scripts/activate
python -m pip install fiscaliza
```

Caso não desejar utilizar um ambiente virtual basta rodar:
`pip install fiscaliza`

## Como utilizar

O `Fiscaliza` da Anatel é um sistema web que utiliza a plataforma Redmine. É basicamente um softaware de gerenciamento e atividades.

O Fiscaliza possui 2 módulos para os fiscais utilizarem:
* Módulo de Teste ( Homologação ) : "https://sistemashm.anatel.gov.br/fiscaliza"
* Módulo de Produção: "https://sistemas.anatel.gov.br/fiscaliza/"

Ambos módulos precisam de um usuário e senha com acesso autorizado, em geral somente servidores da Anatel.

O Escopo desse módulo basicamente está encapsulado em 3 funções básicas:
* detalhar_inspecao - **Retorna as informações do estado atual da Issue ( Inspeção )**
* validar_dicionario - **Para dado dicionário de dados, formata-o para a API do Redmine**
* relatar_inspecao - **Atualiza e Issue com o dicionário de dados passado**

A terceira função `relatar_inspecao` possui escopo limitado. Atualmente somente é formatado e relatado Inspeções ( Issue ) to tipo "Uso do Espectro - Monitoração". Essa inspeção é a principal utilizada para as atividades de monitoração da Anatel e foi o que motivou a criação da presente biblioteca. Outras inspeções possuem campos distintos e assim exigem formatação distinta. Versões futuras poderão abarcar o relato de diferentes inspeções. 

### Detalhar Inspeção

In [ ]:
#hide
show_doc(detalhar_issue)

<h4 id="detalhar_issue" class="doc_header"><code>detalhar_issue</code><a href="https://github.com/ronaldokun/fiscaliza/tree/master/fiscaliza/redmine.py#L501" class="source_link" style="float:right">[source]</a></h4>

> <code>detalhar_issue</code>(**`inspecao`**:"Número da Inspeção a ser relatada", **`login`**:"Login Anatel do Usuário"=*`None`*, **`senha`**:"Senha Utilizada nos Sistemas Interativos da Anatel"=*`None`*, **`fiscaliza`**:"Objeto Redmine logado, opcional ao login e senha"=*`None`*, **`teste`**:"Indica se o relato será de teste"=*`True`*)

Recebe número da inspeção `inspecao`, o login e senha ou opcionalmente objeto Redmine logado `fiscaliza`
inspecao: str - Número da Inspeção a ser relatada
login: str - Login Anatel do Usuário
senha: str - Senha Utilizada nos Sistemas Interativos da
fiscaliza: Redmine - Objeto Redmine logado, opcional ao login e senha
teste: bool - Caso verdadeiro o Fiscaliza de Teste ( Homologação ) é utilizado

Returns:
    dict: Retorna um dicionário com a Situação Atual e campos preenchidos da Inspeção

Vamos exemplificar a Inspeção de Teste `57824`

In [ ]:
login = getuser()
senha = getpass()

 ············


In [ ]:
issue = '72088'
detalhes = detalhar_issue(issue, login, senha)
console.print(detalhes)

{
    'id': '72088',
    'subject': 'ACAO_GR01_2021_0688',
    'status': 'Em andamento',
    'priority': 'Normal',
    'start_date': '',
    'due_date': '2021-07-31',
    'description': '',
    'Data_de_inicio_efetivo': datetime.date(2021, 7, 19),
    'Ano': '2021',
    'Tema': [],
    'Subtema': [],
    'Tipo_de_Fiscalizacao': 'Comum',
    'Diretriz': [],
    'Macrotema_uso_SFI': '',
    'Coordenacao': '',
    'Numero_Sei_do_Processo': '{"numero"=>"53554.000013/2021-11", "link_acesso"=>"https://sei
hm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1964149"}',
    'Fiscal_Responsavel': '',
    'Horas_de_Preparacao': '',
    'Horas_de_Deslocamento': '',
    'Horas_de_Execucao': '',
    'Horas_de_elaboracao_documental': '',
    'Horas_de_Conclusao': '',
    'Origem_da_Acao': 'Solicitação de Fiscalização',
    'No_SEI_Relatorio_Fiscalizacao': '',
    'No_SEI_do_Despacho_Aprovacao': '',
    'Agrupamento': '',
    'Data_da_demanda_original': '',
    'Centralizador_Regional': '',
    'AppFiscaliza': '0',
    'Users': [
        'Alexandre Elias de Andrade Oliveira',
        'Alexandre Freitas de Lima',
        'Alexandre Inacio',
        'Alexandre Junzo Hamada',
        'Alfredo de Andrade Filho',
        'Ananias Pereira',
        'Antonio Carlos Cardoso de Mello',
        'Aparecido Sebastiao da Silva',
        'Arthur Pisaruk',
        'Carlos Augusto de Carvalho',
        'Carlos Eduardo Guimaraes Silveira',
        'Carlos da Paixao Filho',
        'Celio Yukio Takahashi',
        'Celso Luiz Maximino',
        'Diogo Caldeira',
        'Ediceu Beraldi',
        'Eduardo Narkevicius',
        'Elcio Maehara',
        'Eustaquio Lages Duarte',
        'Gauber Albuquerque',
        'Gilson Ponce Ayres Filho',
        'Helio Lopes de Carvalho Filho',
        'Higor da Paz Melo',
        'Hugo Santana Lima',
        'Humberto Barbosa Vinagre',
        'Jamilson Evangelista',
        'Joao Yokoyama',
        'Joaquim Miranda',
        'José Antônio S. Sanches',
        'Julio Cesar de Assis Santos',
        'Kiyotomo Kawamura',
        'Laert Calil Junior',
        'Lannei Vilela Moraes',
        'Luis Lagomes',
        'Luiz Vinicios Mielniczuk Seelig',
        'Marcelo Augusto Scacabarozi',
        'Marcelo Vaz Netto',
        'Marcio Colazingari',
        'Marcio Costa',
        'Marcio Rodrigues Maciel',
        'Marcos Antônio Rodrigues',
        'Marcos Juliano Valim da Silva',
        'Maria Teresa Flosi Garrafa',
        'Mario Augusto Volpini',
        'Murilo Amaro',
        'Osnir Lopes',
        'Paulo Diogo Costa',
        'Pedro Arai',
        'Renato Sadao Kushioyada',
        'Ricardo Santos Marques',
        'Ricardo da Silva e Souza',
        'Roberto Carlos Soares Campos',
        'Roberto Ferreira dos Santos',
        'Roberto Takata',
        'Rodrigo Barbosa de Paula',
        'Rogerio Zambotto',
        'Ronaldo da Silva Alves Batista',
        'Sergio Pereira',
        'Thiago Silva',
        'Thomaz Honma Ishida',
        'Vinicius Paiva de Oliveira',
        'Vitor Zelada',
        'Wellington Devechi Piauilino',
        'Wladimir Senise'
    ],
    'Classe_da_Inspecao': '',
    'Tipo_de_Inspecao': '',
    'UF_Municipio': '',
    'Servicos_da_Inspecao': '',
    'Atualizado': 'Atualizado por Ronaldo da Silva Alves Batista em 02/08/2021 às 09:53:09'
}

In [ ]:
issue = '72120'
detalhes = detalhar_issue(issue, login, senha, teste=True)
console.print(detalhes)

{
    'id': '72120',
    'subject': 'INSP_GR01_2021_0702',
    'status': 'Rascunho',
    'priority': 'Normal',
    'start_date': '',
    'due_date': '',
    'description': '',
    'Data_de_inicio_efetivo': '',
    'Classe_da_Inspecao': '',
    'Tipo_de_Inspecao': '',
    'Ano': '2021',
    'Coordenacao': '',
    'Numero_Sei_do_Processo': '{"numero"=>"53554.000013/2021-11", "link_acesso"=>"https://sei
hm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1964149"}',
    'Fiscal_Responsavel': '',
    'Fiscais': [],
    'Entidade_da_Inspecao': [],
    'UF_Municipio': [],
    'Servicos_da_Inspecao': [
        '026 - COLETIVO - RADIOENLACES ASSOCIADOS AO SERVIÇO DE CIRCUITO ESPECIALIZADO',
        '027 - COLETIVO - RADIOENLACES ASSOCIADOS AO SERVIÇO MÓVEL ESPECIALIZADO',
        '251 - RESTRITO - AUXILIAR RADIODIF.- TRANSMISS. DE PROGRAMAS',
        '029 - COLETIVO - RADIOENLACES ASSOCIADOS AO SERVIÇO LIMITADO ESPECIALIZADO',
        '015 - COLETIVO - RADIOENLACES ASSOCIADOS A SISTEMA DE SATÉLITES'
    ],
    'Horas_de_Preparacao': '',
    'Horas_de_Deslocamento': '',
    'Horas_de_Execucao': '',
    'Horas_de_Conclusao': '',
    'SAV': '',
    'PCDP': '',
    'Procedimentos': [],
    'Agrupamento': '',
    'AppFiscaliza': '0',
    'Relatorio_de_Monitoramento': '',
    'id_ACAO': 72088,
    'nome_ACAO': 'ACAO_GR01_2021_0688',
    'descricao_ACAO': '',
    'Users': [
        'Alexandre Elias de Andrade Oliveira',
        'Alexandre Freitas de Lima',
        'Alexandre Inacio',
        'Alexandre Junzo Hamada',
        'Alfredo de Andrade Filho',
        'Ananias Pereira',
        'Antonio Carlos Cardoso de Mello',
        'Aparecido Sebastiao da Silva',
        'Arthur Pisaruk',
        'Carlos Augusto de Carvalho',
        'Carlos Eduardo Guimaraes Silveira',
        'Carlos da Paixao Filho',
        'Celio Yukio Takahashi',
        'Celso Luiz Maximino',
        'Diogo Caldeira',
        'Ediceu Beraldi',
        'Eduardo Narkevicius',
        'Elcio Maehara',
        'Eustaquio Lages Duarte',
        'Gauber Albuquerque',
        'Gilson Ponce Ayres Filho',
        'Helio Lopes de Carvalho Filho',
        'Higor da Paz Melo',
        'Hugo Santana Lima',
        'Humberto Barbosa Vinagre',
        'Jamilson Evangelista',
        'Joao Yokoyama',
        'Joaquim Miranda',
        'José Antônio S. Sanches',
        'Julio Cesar de Assis Santos',
        'Kiyotomo Kawamura',
        'Laert Calil Junior',
        'Lannei Vilela Moraes',
        'Luis Lagomes',
        'Luiz Vinicios Mielniczuk Seelig',
        'Marcelo Augusto Scacabarozi',
        'Marcelo Vaz Netto',
        'Marcio Colazingari',
        'Marcio Costa',
        'Marcio Rodrigues Maciel',
        'Marcos Antônio Rodrigues',
        'Marcos Juliano Valim da Silva',
        'Maria Teresa Flosi Garrafa',
        'Mario Augusto Volpini',
        'Murilo Amaro',
        'Osnir Lopes',
        'Paulo Diogo Costa',
        'Pedro Arai',
        'Renato Sadao Kushioyada',
        'Ricardo Santos Marques',
        'Ricardo da Silva e Souza',
        'Roberto Carlos Soares Campos',
        'Roberto Ferreira dos Santos',
        'Roberto Takata',
        'Rodrigo Barbosa de Paula',
        'Rogerio Zambotto',
        'Ronaldo da Silva Alves Batista',
        'Sergio Pereira',
        'Thiago Silva',
        'Thomaz Honma Ishida',
        'Vinicius Paiva de Oliveira',
        'Vitor Zelada',
        'Wellington Devechi Piauilino',
        'Wladimir Senise'
    ],
    'Atualizado': 'Atualizado por Ronaldo da Silva Alves Batista em 02/08/2021 às 11:00:49'
}

Agora vamos testar o retorno de informações para um Inspeção no Fiscaliza de produção `teste=False`.

### Validação de Dados e Formatação para submeter à API
A função anterior somente retorna as informações já constantes em dada Inspeção ( Issue ) do Fiscaliza. Para alterarmos ou atualizarmos dada inspeção, precisamos passar um dicionário de dados ou um caminho para um arquivo `.json` ou pickle `.pkl` onde conste esse dicionário de dados serializado. O exemplo seguinte mostra um dicionário de dados com as informações básicas constantes de uma monitoração e a formatação que elas são validadas:

In [ ]:
d = {}

d['Classe_da_Inspecao'] = 'Técnica' # str

d['Tipo_de_Inspecao'] = 'Uso do Espectro - Monitoração' #str

d['description'] = '''Atendimento da Denúncia AC202010}213075425 (6104512), 
verificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus harmônicos, 
além da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.''' #str

d['Fiscal_Responsavel'] = 'Ronaldo da Silva Alves Batista' #str

d['Fiscais'] = ['Ronaldo da Silva Alves Batista', 'Paulo Diogo Costa', 'Mario Augusto Volpini'] #string ou lista de strings

# Windows
d['Html'] = r'D:\OneDrive - ANATEL\Monitoramento\Processos\53504.0005432021-55\Sorocaba.html' #str

#ou 

#Unix d['Html'] = '/d/OneDrive - ANATEL/Monitoramento/53504.0005432021-55/Guarulhos.html' #str
            
d['Gerar_Relatorio'] = 1 # int 0 ou 1

d['Frequencia_Inicial']  = 54 #int ou float

d['Unidade_da_Frequencia_Inicial'] = 'MHz' #string

d['Frequencia_Final'] = 700 #int ou float

d['Unidade_da_Frequencia_Final'] = 'MHz' #string

d['start_date'] = '2021-03-19' #YYYY-MM-DD #string nesse formato

d['due_date'] = '2021-12-31'  #YYYY-MM-DD #string nesse formato

d['UF_Municipio'] = "SP/São Paulo" # string ou Lista de Strings: ["SP/São Paulo", "SP/Sorocaba"]

d['Servicos_da_Inspecao'] = SERVICOS.keys() # String ou Lista de Strings

d['Qnt_de_emissoes_na_faixa'] = 12 # int

d['Emissoes_nao_autorizadas'] = 70 # int

d['Horas_de_Preparacao'] = 2 # int

d['Horas_de_Deslocamento'] = 0 # int

d['Horas_de_Execucao'] = 32 # int

d['Horas_de_Conclusao'] = 6 # int

d['Latitude'] = -22.94694 # float

d['Longitude'] = -43.21944 # float

d['Uso_de_PF'] = 'Não se aplica PF - uso apenas de formulários' # string

d['Acao_de_risco_a_vida_criada'] = 'Não' # string Sim | Não

d['Impossibilidade_acesso_online'] = '0' # string '0' | '1'

d['Reservar_Instrumentos'] = '0' #string '0' = Não | '1' = 'Sim'

d['Utilizou_algum_instrumento'] = '0'

#d['Reserva de Instrumentos'] = '' 

d['notes'] = "Não foi constatada irregularidade no Período monitorado" # string

# No caso de uma tabela, string formatada como csv

d['notes'] = ["""Faixa, Classe Especial, Classe A, Classe B, Classe C
                VHF-L,0,5,7,5
                VHF-H,0,12,1,0
                UHF,1,1,2,4
                FM,5,1,0,0
                RADCOM,0,0,0,0
                Outorgadas com indícios de irregularidades,1,2,3,4
              """,
              """[{"Banda":"87.800 - 108.000 MHz","Sinais":35,"Licenciados":31,"NaoLicenciados":4,"NaoLicenciaveis":0},
                  {"Banda":"470.000 - 698.000 MHz","Sinais":19,"Licenciados":19,"NaoLicenciados":0,"NaoLicenciaveis":0}]
              """]

In [ ]:
f = auth_user(login, senha, teste=True)
insp = f.issue.get(issue, include=["relations", "attachments"])

In [ ]:
f.issue.update(issue, custom_fields=[{'id': 535, 'value': ''}], status='Relatada')

In [ ]:
list(list(insp.custom_fields)[-3])

[('id', 213), ('name', 'Agrupamento'), ('value', '')]

In [ ]:
for i in insp.journals:
    print(getattr(i, 'notes', None))

In [ ]:
# d = {}

# d['Classe_da_Inspecao'] = 'Técnica' # str

# d['Tipo_de_Inspecao'] = 'Uso do Espectro - Monitoração' #str

# d['description'] = '''[PRD-M21]SP Atendimento ao Plano de Ação de Fiscalização GR01FI3 (6595798). 
# Deverá ser fornecido um único relatório de atividades detalhado com análise da utilização do espectro de todas as faixas e, eventualmente, emissões fora de faixa, espúrios, harmônicos e produtos de intermodulação que afetem outras faixas de frequência. 
# Os resultados consolidados devem ser colocados no campo "Notas" do Fiscaliza conforme modelo definido no item 6.5.1.3 do Plano de Ação de Fiscalização GR01FI3 (6595798).''' #str

# d['Fiscal_Responsavel'] = 'Paulo Diogo Costa' #str

# d['Fiscais'] = ['Paulo Diogo Costa'] #string ou lista de strings

# # Windows
# d['Html'] = 'D:\\OneDrive - ANATEL\\Monitoramento\\Processos\\Diogo\\ETM\\Report_2021.06.14_T16.00.06_FinalVersion.html' #str

# #ou 

# #Unix d['Html'] = '/d/OneDrive - ANATEL/Monitoramento/53504.0005432021-55/Guarulhos.html' #str
            
# d['Gerar_Relatorio'] = 0 # int 0 ou 1

# d['Frequencia_Inicial']  = 54 #int ou float

# d['Unidade_da_Frequencia_Inicial'] = 'MHz' #string

# d['Frequencia_Final'] = 800 #int ou float

# d['Unidade_da_Frequencia_Final'] = 'MHz' #string

# d['start_date'] = '2021-03-15' #YYYY-MM-DD #string nesse formato

# d['due_date'] = '2021-09-30'  #YYYY-MM-DD #string nesse formato

# d['UF_Municipio'] = "SP/Presidente Prudente" # string ou Lista de Strings: ["SP/São Paulo", "SP/Sorocaba"]

# d['Servicos_da_Inspecao'] = ['230', '231', '247', '801'] # String ou Lista de Strings

# d['Qnt_de_emissoes_na_faixa'] = 20 # int

# d['Emissoes_nao_autorizadas'] = 1 # int

# d['Horas_de_Preparacao'] = 4 # int

# d['Horas_de_Deslocamento'] = 6 # int

# d['Horas_de_Execucao'] = 16 # int

# d['Horas_de_Conclusao'] = 8 # int

# d['Latitude'] = -22.117028 # float

# d['Longitude'] = -51.405222 # float

# d['Uso_de_PF'] = 'Não se aplica PF - uso apenas de formulários' # string

# d['Acao_de_risco_a_vida_criada'] = 'Não' # string Sim | Não

# d['Impossibilidade_acesso_online'] = '0' # string '0' | '1'

# d['Reservar_Instrumentos'] = '0' #string '0' = Não | '1' = 'Sim'

# d['Utilizou_algum_instrumento'] = '0'

# #d['Reserva de Instrumentos'] = '' 

# # d['notes'] = "Não foi constatada irregularidade no Período monitorado" # string

# # # No caso de uma tabela, string formatada como csv

# d['notes'] = ["""Faixa, Classe Especial, Classe A, Classe B, Classe C
#                 VHF-L,0,0,0,0
#                 VHF-H,0,0,0,0
#                 UHF,1,3,2,6
#                 FM,0,6,1,0
#                 RADCOM,0,0,0,0
#                 Outorgadas com indícios de irregularidades,0,4,0,4
#               """]

In [ ]:
# json.dump(d,  Path("D:/OneDrive - ANATEL/Code/fiscaliza/fiscaliza/files/data_pd.json").open('w'))

A API do Redmine possui formatos específicos de como esses campos devem ser submetidos, validar os formatos acima e fazer essa formatação exigida pela API do Redmine é o papel da função `validar_dicionario`.

In [ ]:
#hide
show_doc(validar_dicionario)

In [ ]:
dados = validar_dicionario(d, issue, login, senha, teste=True)

In [ ]:
# json.dump(dados,  Path("D:/OneDrive - ANATEL/Code/fiscaliza/fiscaliza/files/dados_pd.json").open('w'))

In [ ]:
for k,v in dados.items():
    if k == "Html":
        v = str(v)
        v = v[:100] + '\n...\n' + v[-100:]
    console.print(f'[blue]{k} [red]-> [green]{v}')

Classe_da_Inspecao -> {'id': 89, 'value': '{"valor":"Técnica","texto":"Técnica"}'}

Tipo_de_Inspecao -> {'id': 2, 'value': '{"valor":"Uso do Espectro - Monitoração","texto":"Uso
do Espectro - Monitoração"}'}

description -> Atendimento da Denúncia AC202010}213075425 (6104512), 
verificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus 
harmônicos, 
além da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.

Fiscal_Responsavel -> {'id': 25, 'value': 887}

Fiscais -> {'id': 26, 'value': [887, 607, 165]}

...

Acao_de_risco_a_vida_criada -> {'id': 154, 'value': 'Não'}

Impossibilidade_acesso_online -> {'id': 450, 'value': '0'}

Reservar_Instrumentos -> {'id': 596, 'value': '0'}

Utilizou_algum_instrumento -> {'id': 598, 'value': '0'}

notes -> |_.  Faixa                                      |_.   Classe Especial |_.   Classe A
|_.   Classe B |_.   Classe C |
|<. VHF-L                                       |>.                 0 |>.          5 |>.     
7 |>.          5 |
|<. VHF-H                                       |>.                 0 |>.         12 |>.     
1 |>.          0 |
|<. UHF                                         |>.                 1 |>.          1 |>.     
2 |>.          4 |
|<. FM                                          |>.                 5 |>.          1 |>.     
0 |>.          0 |
|<. RADCOM                                      |>.                 0 |>.          0 |>.     
0 |>.          0 |
|<. Outorgadas com indícios de irregularidades  |>.                 1 |>.          2 |>.     
3 |>.          4 |
|_.  Banda                 |_.   Sinais |_.   Licenciados |_.   Não Licenciados |_.   Não 
Licenciáveis |
|<. 87.800 - 108.000 MHz   |>.       35 |>.            31 |>.                 4 |>.          
0 |
|<. 470.000 - 698.000 MHz  |>.       19 |>.            19 |>.                 0 |>.          
0 |

### Relatar Inspeção
A função a seguir é a mais importante do módulo porque ela de fato altera os dados na plataforma Fiscaliza.

In [ ]:
#hide
show_doc(relatar_inspecao)

In [ ]:
relatar_inspecao(issue, login, senha, dados=dados, teste=True, parar_em="Relatada", substituir_relatorio=True)

Usuário Autenticado com Sucesso 👍

Inspeção 72120 vinculada à Ação 72088

Output()

✅ Estado Atual: Rascunho

👩‍💻  A inspeção será atualizada até a situação Relatada

...

✨ Inspeção 72120 atualizada para Relatando

Output()

✒ Assine o Relatório de Monitoramento e chame a função novamente ❗

Output()

{'id': '72120',
 'subject': 'INSP_GR01_2021_0702',
 'status': 'Relatando',
 'priority': 'Normal',
 'start_date': '2021-03-19',
 'due_date': '2021-12-31',
 'description': 'Atendimento da Denúncia AC202010}213075425 (6104512), \r\nverificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus harmônicos, \r\nalém da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.',
 'Data_de_inicio_efetivo': datetime.date(2021, 8, 2),
 'Classe_da_Inspecao': 'Técnica',
 'Tipo_de_Inspecao': 'Uso do Espectro - Monitoração',
 'Ano': '2021',
 'Coordenacao': '',
 'Numero_Sei_do_Processo': '{"numero"=>"53554.000013/2021-11", "link_acesso"=>"https://seihm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1964149"}',
 'Fiscal_Responsavel': 'Ronaldo da Silva Alves Batista',
 'Fiscais': ['Ronaldo da Silva Alves Batista',
  'Paulo Diogo Costa',
  'Mario Augusto Volpini'],
 'Entidade_da_Inspecao': [],
 'UF_Municipio': ['SP/São Paulo'],
 'Serv

O dicionário de dados possui os dados completos obrigatórios para o relato, a cada etapa os dados são atualizados caso distintos do que já consta no fiscaliza:
* `Rascunho` para `Aguardando Execução`
* `Aguardando Execução` para `Em Andamento`
* `Em Andamento` para `Relatando`
* `Relatando` para `Relatada`

Após o relatório ser assinado basta chamar a função com os mesmos argumentos para que a etapa final seja realizada.

In [ ]:
relato = relatar_inspecao(issue, login, senha, dados=dados, parar_em="Relatada")

Usuário Autenticado com Sucesso 👍

Inspeção 72120 vinculada à Ação 72088

Output()

✅ Estado Atual: Relatando

 ⚠ Já existe um Relatório de Monitoramento criado, esse campo não será atualizado ⚠

...

Output()

✨ Inspeção 72120 atualizada para Relatando

Output()

😎 Inspeção 72120 atualizada para Relatada

Output()

Os mesmos passos acima podem ser efetuados no **Fiscaliza** de produção, bastando passar o argumento `teste=False` nas funções acima

In [ ]:
inspecao = '51849'
detalhes = detalhar_issue(inspecao, login, senha, teste=False)
console.print(detalhes)